In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model, layers, optimizers

import numpy as np
import pandas as pd
import os
import random

In [2]:
np.random.seed(2022)
random.seed(2022)
tf.random.set_seed(2022)
os.environ['PYTHONHASHSEED'] = '0'

In [3]:
data = pd.read_csv('./creditcard.csv')

In [4]:
data.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,3.918649e-15,5.682686e-16,-8.761736e-15,2.811118e-15,-1.552103e-15,2.040130e-15,-1.698953e-15,-1.893285e-16,-3.147640e-15,...,1.473120e-16,8.042109e-16,5.282512e-16,4.456271e-15,1.426896e-15,1.701640e-15,-3.662252e-16,-1.217809e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [5]:
from sklearn.preprocessing import RobustScaler

rob_scaler = RobustScaler()

data['amount'] = rob_scaler.fit_transform(data['Amount'].values.reshape(-1,1))
data['time'] = rob_scaler.fit_transform(data['Time'].values.reshape(-1,1))

data.drop(['Amount', 'Time'], axis=1, inplace=True)

In [6]:
data1 = data.sample(frac=1)

fraud_data = data1.loc[data1['Class'] == 1]
non_fraud_data = data1.loc[data1['Class'] == 0][:492]

In [7]:
balanced_data = pd.concat([fraud_data, non_fraud_data])

new_data = balanced_data.sample(frac=1, random_state=42)

In [9]:
X = new_data.drop('Class', axis=1)
y = new_data['Class']

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

In [13]:
model = keras.Sequential([
    layers.Dense(X_train.shape[1], input_shape=(X_train.shape[1], ), activation='relu'),
    layers.Dense(2, activation='softmax')
])

In [14]:
model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [15]:
model.fit(X_train, y_train, validation_split=0.2, batch_size=20, epochs=20, shuffle=True, verbose=2)

Epoch 1/20
32/32 - 0s - loss: 0.7101 - accuracy: 0.5644 - val_loss: 0.6069 - val_accuracy: 0.6329 - 320ms/epoch - 10ms/step
Epoch 2/20
32/32 - 0s - loss: 0.4366 - accuracy: 0.7520 - val_loss: 0.4215 - val_accuracy: 0.7468 - 31ms/epoch - 970us/step
Epoch 3/20
32/32 - 0s - loss: 0.3090 - accuracy: 0.8808 - val_loss: 0.3363 - val_accuracy: 0.8418 - 30ms/epoch - 938us/step
Epoch 4/20
32/32 - 0s - loss: 0.2445 - accuracy: 0.9285 - val_loss: 0.2882 - val_accuracy: 0.8797 - 30ms/epoch - 938us/step
Epoch 5/20
32/32 - 0s - loss: 0.2051 - accuracy: 0.9428 - val_loss: 0.2633 - val_accuracy: 0.8861 - 30ms/epoch - 938us/step
Epoch 6/20
32/32 - 0s - loss: 0.1784 - accuracy: 0.9491 - val_loss: 0.2432 - val_accuracy: 0.8987 - 31ms/epoch - 970us/step
Epoch 7/20
32/32 - 0s - loss: 0.1599 - accuracy: 0.9555 - val_loss: 0.2305 - val_accuracy: 0.8924 - 30ms/epoch - 938us/step
Epoch 8/20
32/32 - 0s - loss: 0.1455 - accuracy: 0.9555 - val_loss: 0.2220 - val_accuracy: 0.8987 - 31ms/epoch - 970us/step
Epoch 9/

In [17]:
under_test_loss, under_test_acc = model.evaluate(X_test, y_test, verbose=2)

7/7 - 0s - loss: 0.1564 - accuracy: 0.9289 - 15ms/epoch - 2ms/step
